In [2]:
### Checking the data

In [3]:
pip install nltk


Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from nltk.tag import pos_tag


In [5]:
df= pd.read_csv("wine_dataset.csv")

In [6]:
df

,id,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,NaN,NaN,Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
129967,129967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
129968,129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
129969,129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss


In [7]:
country_values = df.groupby('country')['country'].count().sort_values(ascending=False)

In [8]:
country_values[:15]

country
US              54504
France          22093
Italy           19540
Spain            6645
Portugal         5691
Chile            4472
Argentina        3800
Austria          3345
Australia        2329
Germany          2165
New Zealand      1419
South Africa     1401
Israel            505
Greece            466
Canada            257
Name: country, dtype: int64

In [9]:
countries_to_keep = ['US','France', 'Italy','Spain','Portugal','Chile','Argentina', 'Austria', 'Australia','Germany','New Zealand', 'South Africa', 'Israel','Greece','Canada']

In [10]:
filtered_df = df[df['country'].isin(countries_to_keep)]

In [11]:
unique_varieties = filtered_df['variety'].unique()
print(unique_varieties)


['White Blend' 'Portuguese Red' 'Pinot Gris' 'Riesling' 'Pinot Noir'
 'Tempranillo-Merlot' 'Frappato' 'Gewürztraminer' 'Cabernet Sauvignon'
 'Nerello Mascalese' 'Chardonnay' 'Malbec' 'Tempranillo Blend' 'Meritage'
 'Red Blend' 'Merlot' "Nero d'Avola" 'Chenin Blanc' 'Gamay'
 'Sauvignon Blanc' 'Viognier-Chardonnay' 'Primitivo' 'Catarratto'
 'Inzolia' 'Petit Verdot' 'Monica' 'Bordeaux-style White Blend' 'Grillo'
 'Sangiovese' 'Cabernet Franc' 'Champagne Blend'
 'Bordeaux-style Red Blend' 'Aglianico' 'Petite Sirah' 'Touriga Nacional'
 'Carmenère' 'Albariño' 'Petit Manseng' 'Rosé' 'Zinfandel' 'Vernaccia'
 'Rosato' 'Grüner Veltliner' 'Viognier' 'Vermentino' 'Grenache Blanc'
 'Syrah' 'Nebbiolo' 'Shiraz-Cabernet Sauvignon' 'Pinot Blanc'
 'Alsace white blend' 'Barbera' 'Rhône-style Red Blend' 'Portuguese White'
 'Graciano' 'Tannat-Cabernet' 'Sauvignon' 'Sangiovese Grosso' 'Torrontés'
 'Prugnolo Gentile' 'G-S-M' 'Verdejo' 'Fumé Blanc' 'Pinot Bianco'
 'Bonarda' 'Shiraz' 'Montepulciano' 'Moscato' 

In [12]:
len(filtered_df['variety'].unique())

647

In [13]:
len(filtered_df['winery'].unique())

16450

--- Data Cleaning

In [15]:
import nltk
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mercedes/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [71]:
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('N'):
        return 'n'
    elif treebank_tag.startswith('V'):
        return 'v'
    elif treebank_tag.startswith('R'):
        return 'r'
    elif treebank_tag.startswith('J'):
        return 'a'
    else:
        return 'n'

def cleaning(sentence):


    sentence = sentence.strip() ## remove whitespaces
    sentence = sentence.lower() ## lowercase
    sentence = ''.join(char for char in sentence if not char.isdigit()) ## remove numbers


    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') ## remove punctuation

    tokenized_sentence = word_tokenize(sentence) ## tokenize
    stop_words = set(stopwords.words('english')) ## define stopwords

    tokenized_sentence_cleaned = [ ## remove stopwords
        w for w in tokenized_sentence if not w in stop_words
    ]

    lemmatized = [
        WordNetLemmatizer().lemmatize(word, pos=get_wordnet_pos(pos_tag))
        for word, pos_tag in pos_tag(tokenized_sentence_cleaned)
    ]


    cleaned_sentence = ' '.join(word for word in lemmatized)

    return cleaned_sentence

In [52]:
df["processed_description"] = df["description"].apply(cleaning)

In [72]:
tfidf_vectorizer = TfidfVectorizer(min_df = 500)

# Fit and transform the processed descriptions
tfidf_matrix = tfidf_vectorizer.fit_transform(df['processed_description'])

In [73]:
vectorized_documents = pd.DataFrame(tfidf_matrix.toarray(),columns = tfidf_vectorizer.get_feature_names_out())
vectorized_documents

,accent,accessible,accompany,acid,acidic,acidity,across,add,additional,aftertaste,...,yeasty,yellow,yet,youll,young,youthful,zest,zesty,zin,zinfandel
0,0.0,0.0,0.0,0.0,0.0,0.122764,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.136283,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.141321,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.277392,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,0.0,0.0,0.0,0.0,0.0,0.148925,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.276343,0.0,0.0,0.0,0.0,0.0,0.0,0.0
129967,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
129968,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
129969,0.0,0.0,0.0,0.0,0.0,0.127838,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [74]:
list(vectorized_documents.columns)

['accent',
 'accessible',
 'accompany',
 'acid',
 'acidic',
 'acidity',
 'across',
 'add',
 'additional',
 'aftertaste',
 'age',
 'air',
 'alcohol',
 'allow',
 'almond',
 'almost',
 'along',
 'alongside',
 'already',
 'also',
 'although',
 'always',
 'amount',
 'ample',
 'anise',
 'another',
 'appeal',
 'appear',
 'appellation',
 'appetizing',
 'apple',
 'approachable',
 'apricot',
 'apéritif',
 'aroma',
 'aromas',
 'aromatic',
 'aromatics',
 'around',
 'asian',
 'assertive',
 'astringency',
 'astringent',
 'attractive',
 'austere',
 'away',
 'back',
 'backbone',
 'background',
 'bacon',
 'bake',
 'baking',
 'balance',
 'balanced',
 'balsamic',
 'banana',
 'barbecue',
 'barrel',
 'base',
 'basic',
 'bean',
 'beautiful',
 'beautifully',
 'become',
 'beef',
 'begin',
 'behind',
 'bell',
 'benefit',
 'berry',
 'best',
 'beyond',
 'big',
 'bit',
 'bite',
 'bitter',
 'bitterness',
 'black',
 'blackberry',
 'blackcherry',
 'blackcurrant',
 'blackfruit',
 'blackskinned',
 'blanc',
 'blend',
 

In [75]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer_n_gram = CountVectorizer(ngram_range = (2,3), min_df=500) # BI-GRAMS
descriptions_vectorized_n_gram = count_vectorizer_n_gram.fit_transform(df['processed_description'])

# Show the representations in a nice DataFrame
descriptions_vectorized_n_gram = pd.DataFrame(
    descriptions_vectorized_n_gram.toarray(),
    columns = count_vectorizer_n_gram.get_feature_names_out(),

)

# Show the vectorized movies with bigrams
descriptions_vectorized_n_gram

,acidity balance,acidity drink,acidity finish,acidity give,acidity keep,acidity wine,aftertaste drink,age drink,age well,age year,...,wine still,wine structure,wine thats,wood age,would pair,yellow apple,yellow fruit,yellow peach,young wine,zesty acidity
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
129967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
129968,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
129969,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
list(descriptions_vectorized_n_gram.columns)


['acidity balance',
 'acidity drink',
 'acidity finish',
 'acidity give',
 'acidity keep',
 'acidity wine',
 'aftertaste drink',
 'age drink',
 'age well',
 'age year',
 'apple citrus',
 'apple flavor',
 'apple pear',
 'apple skin',
 'aroma flavor',
 'aroma lead',
 'aroma palate',
 'aromas black',
 'aromas come',
 'aromas flavor',
 'aromas lead',
 'bake spice',
 'baking spice',
 'balance acidity',
 'balsamic note',
 'barrel sample',
 'barrel spice',
 'bell pepper',
 'berry aromas',
 'berry cherry',
 'berry flavor',
 'berry fruit',
 'berry plum',
 'black cherry',
 'black currant',
 'black fruit',
 'black olive',
 'black pepper',
 'black plum',
 'black raspberry',
 'black tea',
 'blackberry black',
 'blackberry black currant',
 'blackberry cassis',
 'blackberry cherry',
 'blackberry currant',
 'blackberry flavor',
 'blackberry fruit',
 'blackberry jam',
 'blackfruit flavor',
 'blend cabernet',
 'blend cabernet sauvignon',
 'blend grenache',
 'blend merlot',
 'blend sangiovese',
 'blend s